In [1]:
import os
import glob
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

ModuleNotFoundError: No module named 'gradio'

In [ ]:
try:
    load_dotenv()
    api_key = os.getenv("OpenAI_API_KEY")
except Exception as e:
    raise ValueError("Failed to load OpenAI API key. Please set the OpenAI_API_KEY environment variable.") from e

In [ ]:
content = []
knowledge_folders = glob.glob("../knowledge_base/*")
for folder in knowledge_folders:
    folder_name = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, show_progress=True)
    documents = loader.load()
    for document in documents:
        document.metadata["doc_type"] = folder_name
        content.append(document)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(content)
len(chunks)

In [ ]:
db_name = "vector_db"
embeddings = OpenAIEmbeddings(api_key=api_key)

In [ ]:
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

In [ ]:
total_vectors = vectorstore.index.ntotal
print(f"Total vectors in the vector store: {total_vectors}")

dimension = vectorstore.index.d
print(f"Dimension of the vectors in the vector store: {dimension}")

In [ ]:
MODEL = "gpt-3.5-turbo"

llm = ChatOpenAI(api_key=api_key, temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch()

> Check out this repository for a production level and installable rag mode.
